In [1]:
import torch
from utils import get_predictions
import os
from utils import get_data,preprocess_image,plot_roc_curve
import pandas as pd
from models import model_dict,select_model
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np

c:\Users\PC\Desktop\Diabetic_Retinopathy\utils.py:15: DeprecationWarning: The module `ray.tune.suggest` has been moved to `ray.tune.search` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest` with `ray.tune.search`.
  from ray.tune.suggest.optuna import OptunaSearch
c:\Users\PC\Desktop\Diabetic_Retinopathy\utils.py:15: DeprecationWarning: The module `ray.tune.suggest.optuna` has been moved to `ray.tune.search.optuna` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest.optuna` with `ray.tune.search.optuna`.
  from ray.tune.suggest.optuna import OptunaSearch


In [2]:

dataset_total = pd.read_csv("C:/Users/PC/Desktop/retinopathy_data/data/labels/label.csv")
validation_directiory = "C:/Users/PC/Desktop/retinopathy_data/data/validation_images"
train_test_image_directory = "C:/Users/PC/Desktop/retinopathy_data/data/test_train_images"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

files_and_dirs = os.listdir("C:/Users/PC/Desktop/saved_models/") 



In [3]:
all_predictions = []
weights = []

for items in files_and_dirs:
    model_name, file_extension = os.path.splitext(items)

    _, _, valid_data = get_data(dataset_total, train_test_image_directory, validation_directiory,
                                train_test_sample_size=10, batch_size=16,
                                image_filter=preprocess_image, model=model_name, validation=True)

    state_dict = torch.load(f"C:/Users/PC/Desktop/saved_models/{items}")

    model_instance = select_model(model_name)

    model_instance = model_instance.to(device)

    model_instance.load_state_dict(state_dict)

    weight = model_dict[model_name]["weight"]
    weights.append(weight)

    predictions = get_predictions(model_instance, valid_data, device)
    
    all_predictions.append(predictions)

In [4]:
all_predictions = list(zip(*all_predictions))

ensemble_preds = []
for preds in all_predictions:
    weighted_votes = Counter()
    for pred, weight in zip(preds, weights):
        weighted_votes[pred] += weight
    ensemble_preds.append(weighted_votes.most_common(1)[0][0])

predicted_labels_list = [int(label.item()) for label in ensemble_preds]

true_labels = []
for _, labels in valid_data:
    true_labels.extend(labels.numpy())

In [5]:
def evaluate_classification_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_true, y_pred)

    return {
        'accuracy': accuracy,
        'f1_score': f1,
        'precision': precision,
        'recall': recall,
        'confusion_matrix': conf_matrix
    }



In [9]:
evaluate_classification_metrics(true_labels,predicted_labels_list)

{'accuracy': 0.252,
 'f1_score': 0.31606503700939226,
 'precision': 0.5639642799678198,
 'recall': 0.25199999999999995,
 'confusion_matrix': array([[46, 34, 80, 12,  1],
        [ 4, 10, 16,  6,  0],
        [ 8,  8,  6,  3,  0],
        [ 0,  0,  7,  1,  0],
        [ 2,  2,  4,  0,  0]], dtype=int64)}